# **Importing the Data from Kaggle**

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d salader/dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/slugify/slugify.py", line 9, in <module>
    import unidecode
ModuleNotFoundError: No module named 'unidecode'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 3, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/__init__.py", line 6, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 74, in <module>
    from slugify import slugify
  File "/usr/local/lib/python3.10/dist-packages/slugify/__init__.py", line 2, in <module>
    from .slugify import *
  File "/usr/local/lib/python3.10/dist-packages/slugify/slugify.py"

In [ ]:
! unzip dogs-vs-cats.zip

In [ ]:
import os
import shutil  # Import the shutil module

shutil.rmtree('/content/test')  # Now you can use shutil.rmtree()
shutil.rmtree('/content/train')  # Now you can use shutil.rmtree()

# **CNN Model**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
# preprocessing over the training set

train_data_generation = ImageDataGenerator(rescale=1./255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True,
                                           )

training_set = train_data_generation.flow_from_directory('/content/dogs_vs_cats/train',
                                                         target_size=(299, 299),
                                                         batch_size=16,
                                                         class_mode='binary')

Found 20000 images belonging to 2 classes.


In [ ]:

# preprocessing the testing set
test_data_generation = ImageDataGenerator(rescale=1./255)
testing_set = test_data_generation.flow_from_directory('/content/dogs_vs_cats/test',
                                                       target_size=(299, 299),
                                                       batch_size=16,
                                                       class_mode='binary')

Found 5000 images belonging to 2 classes.


In [ ]:
model = Sequential([
            #layer 1
            Conv2D(filters=64, kernel_size=4, activation='relu', input_shape = (299, 299, 3)),
            MaxPooling2D(pool_size=2),

            #layer 2
            Conv2D(filters=32, kernel_size=3, activation='relu'),
            MaxPooling2D(pool_size=2),
            Dropout(0.2),


            Flatten(),
            Dense(units=128, activation='relu'),
            Dense(units=128, activation='relu'),

            Dense(units=1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.models import load_model


model = load_model('/content/drive/MyDrive/Models/CatsVsDogs/model2.h5')

In [ ]:
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x = training_set, validation_data = testing_set, epochs=5 ,  batch_size=32, callbacks = EarlyStopping(patience=4))

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 423s 336ms/step - accuracy: 0.8144 - loss: 0.4053 - val_accuracy: 0.8024 - val_loss: 0.4180
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 449s 343ms/step - accuracy: 0.8212 - loss: 0.3965 - val_accuracy: 0.8192 - val_loss: 0.4019
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 445s 344ms/step - accuracy: 0.8353 - loss: 0.3749 - val_accuracy: 0.8206 - val_loss: 0.3918
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 417s 332ms/step - accuracy: 0.8419 - loss: 0.3557 - val_accuracy: 0.8240 - val_loss: 0.3901
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 451s 340ms/step - accuracy: 0.8496 - loss: 0.3408 - val_accuracy: 0.8250 - val_loss: 0.3990


In [ ]:
model.save('model3.h5')

In [ ]:
print(training_set.class_indices)
labels = '\n'.join(sorted(training_set.class_indices.keys()))

with open('labels.txt', 'w') as f:
     f.write(labels)

{'cats': 0, 'dogs': 1}


In [ ]:
model.evaluate(testing_set)

313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.8311 - loss: 0.3894


[0.3989962637424469, 0.824999988079071]

In [ ]:
model_weights = load_model('/content/drive/MyDrive/Models/CatsVsDogs/model3.h5')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
def predict_image(filename):
  test_image = image.load_img(filename, target_size = (299, 299))
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  result = model_weights.predict(test_image)
  if result[0][0]  >= 0.5:
    prediction = 'dog'
  else:
    prediction = 'cat'
  return prediction
predict_image("/content/drive/MyDrive/Impo_Media/IMG_20230209_141747.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


'dog'